In [1]:
    #   "element_force": [
    #     -145377944.0594964,
    #     64810376.4682987,
    #     125978543743.83755,
    #     145377944.0594964,
    #     -64810376.4682987,
    #     100857773895.20789
    #   ],

In [2]:
import math
import copy

Fy   = 2.65000e-01
E    = 2.10000e+02
u    = 0.3
G    = E/(2*(1+u))

Column_Data = [-50.05, 7518, 37308, 50, 100]         # Pu, Muy, Muz, Vuy, Vuz - 기둥 Nodal Output, 
Girder_Data = [-50.05, 7518, 7308, 50, 100]        # Pu, Muy, Muz, Vuy, Vuz - Y방향 큰보 Nodal Output,  [145377944.0594964,125978543743.83755,0,0, 64810376.4682987]
Beam_Data = [-50.05, 7518, 7308, 50, 100]          # Pu, Muy, Muz, Vuy, Vuz - Y방향 작은보 Nodal Output
Sub_Column_Data = [0, 20000, 20000, 50, 100]         # Pu, Muy, Muz, Vuy, Vuz - 샛기둥 Nodal Output
                                          
Column_Length = 7000                                                                    # Floor가 있는 층이 있으면 강축방향 Brace를 층에서 잡아줌 / 부재별 Length로 사용가능
Column_Unbraced_Segment = 2                                                             # 기둥에 추가 Lateral Brace 개수 존재 여부 (약축 Unbraced Length 잡아줌, 1은 Lateral Brace 없는 것)
Girder_Length = 6000                                                                    # Girder & Beam 강축방향 Unbraced Length / 부재별 Length로 사용가능
Girder_Unbraced_Segment = 3                                                             # X-Girder 약축방향 Lateral Brace 개수 존재 여부 (약축 Unbraced Length 잡아줌, 1은 Lateral Brace 없는 것, 0은 Continuous Braced)
Beam_Length = 5000                                                                      # Beam 길이
Beam_Attached_Girder_Length = 5500                                                      # Beam이 붙은 Girder의 길이
X_Span = 6000                                                                           # Column 약축방향쪽 Span
Y_Span = 7000

K = 1
PhiP = 0.9
PhiM = 0.9

In [3]:
def make_forceDict(data):
    keys = ["Pu", "Muy", "Muz", "Vuy", "Vuz"]
    return dict(zip(keys, data))

col_forceDict = make_forceDict(Column_Data)
girder_forceDict = make_forceDict(Girder_Data)
beam_forceDict = make_forceDict(Beam_Data)
beam_forceDict

{'Pu': -50.05, 'Muy': 7518, 'Muz': 7308, 'Vuy': 50, 'Vuz': 100}

In [4]:
# 0             1     2    3    4     5     6    7    8    9    10   11   12   13   14   15  16  17 18   19  20 21 22  23  24
# Section Name, Area, Asy, Asz, Ybar, Zbar, Qyb, Qzb, Syy, Szz, Zyy, Zzz, Iyy, Izz, Iyz, ry, rz, J, Cwp, SW, D, W, tw, tf, radi
propKeys = ["Section_Name", "Area", "Asy", "Asz", "Ybar", "Zbar", "Qyb", "Qzb", "Syy", "Szz", "Zyy", "Zzz", "Iyy", "Izz", "Iyz", "ry", "rz", "J", "Cwp", "SW", "D", "W", "tw", "tf", "radi"]

Section = [
['UB-203x102x23',	      2.90000e+03,	1.25984e+03,	1056.64,	5.08000e+01,	1.01600e+02,	2.18762e+04,	1290.32,	2.06000e+05,	3.21000e+04,	2.32000e+05,	49500,	2.09100e+07,	1.63000e+06,	0,	8.49000e+01,	23.7,	6.87000e+04,	15320900000,	23,	203.2,	101.6,	5.200,	9.3,  7.6],
['UB-203x133x30',	      3.80000e+03,	1.71264e+03,	1302.84,	6.69000e+01,	1.03400e+02,	2.45024e+04,	2237.81,	2.79000e+05,	5.74000e+04,	3.13000e+05,	88000,	2.88800e+07,	3.84000e+06,	0,	8.72000e+01,	31.8,	1.02000e+05,	37332300000,	30,	206.8,	133.8,	6.300,	9.6,  7.6],
['UB-254x146x31',	      3.99000e+03,	1.67528e+03,	1534.15,	7.30500e+01,	1.25750e+02,	3.18780e+04,	2668.15,	3.52000e+05,	6.13000e+04,	3.95000e+05,	94200,	4.42800e+07,	4.48000e+06,	0,	1.05000e+02,	33.5,	8.65000e+04,	66080500000,	31,	251.5,	146.1,	6.100,	8.6,  7.6],
['UB-254x146x37',	      4.74000e+03,	2.12768e+03,	1638.4,	7.32000e+01,	1.28000e+02,	3.74125e+04,	2679.12,	4.33000e+05,	7.80000e+04,	4.85000e+05,	119000,	5.54700e+07,	5.71000e+06,	0,	1.08000e+02,	34.7,	1.54000e+05,	85755600000,	37,	256.0,	146.4,	6.400,	10.9, 7.6],
['UB-305x165x40',	      5.16000e+03,	2.24536e+03,	1853.18,	8.25500e+01,	1.51900e+02,	5.05664e+04,	3407.25,	5.63000e+05,	9.28000e+04,	6.26000e+05,	142000,	8.55100e+07,	7.66000e+06,	0,	1.29000e+02,	38.5,	1.49000e+05,	165075000000,	40,	303.8,	165.1,	6.100,	10.2, 8.9],
['UB-305x165x54',	      6.82000e+03,	3.04688e+03,	2393.93,	8.34000e+01,	1.55450e+02,	5.41472e+04,	3477.78,	7.52000e+05,	1.27000e+05,	8.43000e+05,	195000,	1.16900e+08,	1.06100e+07,	0,	1.31000e+02,	39.4,	3.43000e+05,	234290000000,	54,	310.9,	166.8,	7.700,	13.7, 8.9],
['UB-356x171x51',	      6.46000e+03,	2.62967e+03,	2595.88,	8.57500e+01,	1.77800e+02,	6.03108e+04,	3676.53,	7.96000e+05,	1.13000e+05,	8.95000e+05,	174000,	1.41600e+08,	9.68000e+06,	0,	1.48000e+02,	38.7,	2.36000e+05,	286540000000,	51,	355.6,	171.5,	7.300,	11.5, 10.2],
['UB-457x152x60',	      7.58000e+03,	2.71143e+03,	3637.6,	7.64500e+01,	2.27350e+02,	7.90098e+04,	2922.3,	1.11900e+06,	1.04000e+05,	1.28300e+06,	163000,	2.54500e+08,	7.95000e+06,	0,	1.83000e+02,	32.4,	3.35000e+05,	387232000000,	60,	454.7,	152.9,	8.000,	13.3, 10.2],
['UB-457x191x74',	      9.51000e+03,	3.68300e+03,	4160.52,	9.52500e+01,	2.28600e+02,	9.01089e+04,	4536.28,	1.46200e+06,	1.76000e+05,	1.65900e+06,	273000,	3.34300e+08,	1.67400e+07,	0,	1.87000e+02,	42,	5.22000e+05,	820190000000,	74,	457.2,	190.5,	9.100,	14.5, 10.2],
['UB-457x191x98',	      1.25000e+04,	5.03851e+03,	5328.36,	9.64000e+01,	2.33700e+02,	9.71379e+04,	4646.48,	1.95900e+06,	2.43000e+05,	2.23400e+06,	379000,	4.57700e+08,	2.34700e+07,	0,	1.91000e+02,	43.3,	1.21000e+06,	1176580000000,	98,	467.4,	192.8,	11.400,	19.6, 10.2],
['UB-533x210x82',	      1.05000e+04,	3.67312e+03,	5071.68,	1.04350e+02,	2.64150e+02,	1.05395e+05,	5444.46,	1.79900e+06,	1.92000e+05,	2.05800e+06,	300000,	4.75200e+08,	2.00400e+07,	0,	2.13000e+02,	43.8,	5.15000e+05,	1329290000000,	82,	528.3,	208.7,	9.600,	13.2, 12.7],
['UB-533x210x101',	1.29000e+04,	4.87432e+03,	5850.03,	1.05050e+02,	2.68350e+02,	1.18572e+05,	5517.75,	2.29700e+06,	2.57000e+05,	2.61900e+06,	400000,	6.16500e+08,	2.69600e+07,	0,	2.18000e+02,	45.7,	1.02000e+06,	1817590000000,	101,	536.7,	210.1,	10.900,	17.4, 12.7],
['UB-610x229x101',	1.29000e+04,	4.49131e+03,	6383.32,	1.13800e+02,	3.01100e+02,	1.34316e+05,	6475.22,	2.51800e+06,	2.56000e+05,	2.88700e+06,	401000,	7.58200e+08,	2.91500e+07,	0,	2.42000e+02,	47.5,	7.76000e+05,	2514470000000,	101,	602.2,	227.6,	10.600,	14.8, 12.7],
['UB-610x229x125',	1.59000e+04,	5.98453e+03,	7281.61,	1.14500e+02,	3.05950e+02,	1.52699e+05,	6555.12,	3.21900e+06,	3.43000e+05,	3.67300e+06,	535000,	9.85000e+08,	3.93200e+07,	0,	2.49000e+02,	49.7,	1.54000e+06,	3448550000000,	125,	611.9,	229,	      11.900,	19.6, 12.7],
['UB-610x305x149',	1.90000e+04,	8.00608e+03,	7254.24,	1.52400e+02,	3.04800e+02,	1.89468e+05,	11612.9,	4.09300e+06,	6.11000e+05,	4.57500e+06,	938000,	1.24700e+09,	9.30800e+07,	0,	2.56000e+02,	69.9,	2.01000e+06,	8097540000000,	149,	609.6,	304.8,	11.900,	19.7, 16.5],
['UB-610x305x179',	2.28000e+04,	9.66026e+03,	8706.75,	1.53500e+02,	3.08750e+02,	1.93241e+05,	11781.1,	4.90700e+06,	7.42000e+05,	5.51500e+06,	1143000,	1.51500e+09,	1.14000e+08,	0,	2.58000e+02,	70.8,	3.40000e+06,	10052400000000,	179,	617.5,	307,	      14.100,	23.6, 16.5],
['UB-686x254x125',	1.59000e+04,	5.46480e+03,	7931.43,	1.26500e+02,	3.38950e+02,	1.67983e+05,	8001.13,	3.48100e+06,	3.46000e+05,	3.99400e+06,	542000,	1.18000e+09,	4.38300e+07,	0,	2.72000e+02,	52.4,	1.16000e+06,	4797710000000,	125,	677.9,	253,	      11.700,	16.2, 15.2],
['UB-686x254x170',	2.17000e+04,	8.08328e+03,	10047.1,	1.27900e+02,	3.46450e+02,	1.91980e+05,	8179.2,	4.91600e+06,	5.18000e+05,	5.63100e+06,	811000,	1.70300e+09,	6.63000e+07,	0,	2.80000e+02,	55.3,	3.08000e+06,	7422760000000,	170,	692.9,	255.8,	14.500,	23.7, 15.2],
['UB-762x267x173',	2.20000e+04,	7.68096e+03,	10896.6,	1.33350e+02,	3.81000e+02,	2.13718e+05,	8891.11,	5.38500e+06,	5.14000e+05,	6.19500e+06,	807000,	2.05200e+09,	6.85000e+07,	0,	3.05000e+02,	55.8,	2.67000e+06,	9387790000000,	173,	762.0,	266.7,	14.300,	21.6, 16.7],
['UB-838x292x194',	2.47000e+04,	8.46010e+03,	12358.3,	1.46200e+02,	4.20350e+02,	2.56217e+05,	10687.2,	6.64100e+06,	6.20000e+05,	7.64000e+06,	974000,	2.79200e+09,	9.06600e+07,	0,	3.36000e+02,	60.6,	3.06000e+06,	15202800000000,	194,	840.7,	292.4,	14.700,	21.7, 17.8],
['UB-914x305x224',	2.86000e+04,	9.69065e+03,	14473.8,	1.52050e+02,	4.55150e+02,	2.95578e+05,	11559.6,	8.26800e+06,	7.39000e+05,	9.53300e+06,	1163000,	3.76300e+09,	1.12400e+08,	0,	3.63000e+02,	62.7,	4.22000e+06,	22078300000000,	224,	910.3,	304.1,	15.900,	23.9, 19.1],
['UC-152x152x30',	      3.84000e+03,	1.91635e+03,	1039.5,	7.64500e+01,	7.87500e+01,	1.85303e+04,	2922.3,	2.22000e+05,	7.33000e+04,	2.48000e+05,	112000,	1.74800e+07,	5.60000e+06,	0,	6.80000e+01,	38.2,	1.06000e+05,	30707100000,	30,	157.5,	152.9,	6.600,	9.4,  7.6],
['UC-203x203x46',	      5.88000e+03,	2.98027e+03,	1483.36,	1.01600e+02,	1.01600e+02,	3.35292e+04,	5161.28,	4.49000e+05,	1.51000e+05,	4.97000e+05,	230000,	4.56500e+07,	1.53900e+07,	0,	8.80000e+01,	51.2,	2.22000e+05,	142130000000,	46,	203.2,	203.2,	7.300,	11,   10.2],
['UC-203x203x71',	      9.09000e+03,	4.75635e+03,	2223.77,	1.03100e+02,	1.07950e+02,	3.84999e+04,	5314.8,	7.07000e+05,	2.45000e+05,	8.01000e+05,	373000,	7.63400e+07,	2.53000e+07,	0,	9.20000e+01,	52.8,	8.10000e+05,	249470000000,	71,	215.9,	206.2,	10.300,	17.3, 10.2],
['UC-254x254x73',	      9.29000e+03,	4.80907e+03,	2184.4,	1.27000e+02,	1.27000e+02,	5.66474e+04,	8064.5,	8.95000e+05,	3.06000e+05,	9.90000e+05,	463000,	1.13700e+08,	3.88000e+07,	0,	1.11000e+02,	64.6,	5.75000e+05,	557789000000,	73,	254.0,	254,	      8.600,	14.2, 12.7],
['UC-305x305x97',	      1.23000e+04,	6.25856e+03,	3047.22,	1.52400e+02,	1.53900e+02,	7.89094e+04,	11612.9,	1.44300e+06,	4.77000e+05,	1.58900e+06,	724000,	2.22000e+08,	7.27200e+07,	0,	1.34000e+02,	76.8,	9.11000e+05,	1554350000000,	97,	207.8,	304.8,	9.900,	15.4, 15.2],
['UC-305x305x118',	1.50000e+04,	7.64955e+03,	3742.55,	1.53400e+02,	1.57250e+02,	8.09028e+04,	11765.8,	1.75600e+06,	5.87000e+05,	1.95200e+06,	892000,	2.76100e+08,	9.00600e+07,	0,	1.36000e+02,	77.6,	1.60000e+06,	1970010000000,	118,	314.5,	306.8,	11.900,	18.7, 15.2],
['UC-356x368x129',	1.65000e+04,	8.59366e+03,	3804.92,	1.84150e+02,	1.77800e+02,	1.14677e+05,	16955.6,	2.26600e+06,	7.92000e+05,	2.48500e+06,	1198000,	4.03000e+08,	1.45800e+08,	0,	1.56000e+02,	93.9,	1.54000e+06,	4166660000000,	129,	355.6,	368.3,	10.700,	17.5, 15.2],
['UC-356x368x153',	1.96000e+04,	1.02175e+04,	4561.2,	1.85100e+02,	1.81000e+02,	1.16635e+05,	17131,	2.68700e+06,	9.46000e+05,	2.97000e+06,	1433000,	4.86400e+08,	1.75100e+08,	0,	1.58000e+02,	94.6,	2.52000e+06,	5099160000000,	153,	362.0,	370.2,	12.600,	20.7, 15.2]
      ]

Column_Section =   ['UB-533x210x82',	      1.05000e+04,	3.67312e+03,	5071.68,	1.04350e+02,	2.64150e+02,	1.05395e+05,	5444.46,	1.79900e+06,	1.92000e+05,	2.05800e+06,	300000,	4.75200e+08,	2.00400e+07,	0,	2.13000e+02,	43.8,	5.15000e+05,	1329290000000,	82,	528.3,	208.7,	9.600,	13.2, 12.7]
Girder_Section = ['UB-254x146x37',	      4.74000e+03,	2.12768e+03,	1638.4,	7.32000e+01,	1.28000e+02,	3.74125e+04,	2679.12,	4.33000e+05,	7.80000e+04,	4.85000e+05,	119000,	5.54700e+07,	5.71000e+06,	0,	1.08000e+02,	34.7,	1.54000e+05,	85755600000,	37,	256.0,	146.4,	6.400,	10.9, 7.6]
Beam_Section =     ['UB-203x133x30',	      3.80000e+03,	1.71264e+03,	1302.84,	6.69000e+01,	1.03400e+02,	2.45024e+04,	2237.81,	2.79000e+05,	5.74000e+04,	3.13000e+05,	88000,	2.88800e+07,	3.84000e+06,	0,	8.72000e+01,	31.8,	1.02000e+05,	37332300000,	30,	206.8,	133.8,	6.300,	9.6,  7.6]

In [5]:
def convert_ListToDict(elist, keylist):
    res = dict(zip(keylist, elist))
    return res

def addKeyVal_toDict(dic,k,v):
    dic[k] = v
    return dic
# convert_ListToDict(Section[0], propKeys)
Sections = list(map(lambda x: convert_ListToDict(x, propKeys), Section))

Sections
Column_Section = Sections[10]
Column_Section["forceDict"] = col_forceDict
Girder_Section = Sections[2]
Girder_Section["forceDict"] = girder_forceDict
Beam_Section = Sections[1]
Beam_Section["forceDict"] = beam_forceDict
# Sections

Columns_secDicts = list(map(lambda x: addKeyVal_toDict(x, "forceDict", col_forceDict), Sections))
Columns_secDicts
Girders_SecDicts = list(map(lambda x: addKeyVal_toDict(x, "forceDict", girder_forceDict), Sections))
Girders_SecDicts
Beams_SecDicts = list(map(lambda x: addKeyVal_toDict(x, "forceDict", beam_forceDict), Sections))
Beams_SecDicts

[{'Section_Name': 'UB-203x102x23',
  'Area': 2900.0,
  'Asy': 1259.84,
  'Asz': 1056.64,
  'Ybar': 50.8,
  'Zbar': 101.6,
  'Qyb': 21876.2,
  'Qzb': 1290.32,
  'Syy': 206000.0,
  'Szz': 32100.0,
  'Zyy': 232000.0,
  'Zzz': 49500,
  'Iyy': 20910000.0,
  'Izz': 1630000.0,
  'Iyz': 0,
  'ry': 84.9,
  'rz': 23.7,
  'J': 68700.0,
  'Cwp': 15320900000,
  'SW': 23,
  'D': 203.2,
  'W': 101.6,
  'tw': 5.2,
  'tf': 9.3,
  'radi': 7.6,
  'forceDict': {'Pu': -50.05,
   'Muy': 7518,
   'Muz': 7308,
   'Vuy': 50,
   'Vuz': 100}},
 {'Section_Name': 'UB-203x133x30',
  'Area': 3800.0,
  'Asy': 1712.64,
  'Asz': 1302.84,
  'Ybar': 66.9,
  'Zbar': 103.4,
  'Qyb': 24502.4,
  'Qzb': 2237.81,
  'Syy': 279000.0,
  'Szz': 57400.0,
  'Zyy': 313000.0,
  'Zzz': 88000,
  'Iyy': 28880000.0,
  'Izz': 3840000.0,
  'Iyz': 0,
  'ry': 87.2,
  'rz': 31.8,
  'J': 102000.0,
  'Cwp': 37332300000,
  'SW': 30,
  'D': 206.8,
  'W': 133.8,
  'tw': 6.3,
  'tf': 9.6,
  'radi': 7.6,
  'forceDict': {'Pu': -50.05,
   'Muy': 7518,


In [6]:
# def Calculate_Slenderness_Ratio (List,n,Ly_) : 
#     Lz_ = Ly_/n
#     ry = List[15]
#     rz = List[16]
#     SR = max(K*Ly_/ry, K*Lz_/rz)

#     return SR

def Calculate_Slenderness_Ratio (secDict,n,Ly_) : 
    Lz_ = Ly_/n
    ry = secDict["ry"]
    rz = secDict["rz"]
    SR = max(K*Ly_/ry, K*Lz_/rz)

    return SR

In [7]:
# def Calculate_Tensile_Strength (List) : 
#     Ag = List[1]
#     Pn = Fy*Ag

#     ΦPn = PhiP*Pn

#     return ΦPn 

def Calculate_Tensile_Strength (secDict) : 
    Ag = secDict["Area"]
    Pn = Fy*Ag

    ΦPn = PhiP * Pn

    return ΦPn 

In [8]:
# def Calculate_Flexural_Buckling(List,n,Ly_) : 
#     Lz_ = Ly_/n
#     Ag = List[1]
#     Iyy = List[12]
#     Izz = List[13]
#     ry = List[15]
#     rz = List[16]
#     J = List[17]
#     Cwp = List[18]
#     D = List[20]
#     W = List[21]
#     tw = List[22]
#     tf = List[23]
#     radi = List[24]
#     SR = max(K*Ly_/ry, K*Lz_/rz)
#     Fe1 = (math.pi**2)*E/(SR**2)
#     Fe2 = ((math.pi**2)*E*Cwp/((K*Ly_)**2)+77.2*J)/(Iyy+Izz)
#     Fe = Fe1

#     λrf = 0.56*math.sqrt(E/Fy)
#     λrw = 1.49*math.sqrt(E/Fy)
#     λf = W/(2*tf)
#     λw = (D-2*tf-2*radi)/tw
    
#     if SR <= 4.71*math.sqrt(E/(Fy)) : 
#         Fn = 0.658**(Fy/Fe)*Fy
#     else : 
#         Fn = 0.877*Fe


#     if (λrf > λf) and (λrw > λw) :      # Flexural Buckling of Member without Slender Element
#         ΦPn = PhiP*Fn*Ag

#     elif (λrf < λf) and (λrw > λw) :    # Flexural Buckling of Member with Slender Flange
#         Fel = (1.49*(λrf/λf))**2*Fy
#         if λf <= λrf*math.sqrt(Fy/Fn) : 
#             Ae = Ag
#         else : 
#             be = (W/2)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
#             Ae = Ag-((W-2*be)*tf)*2
        
#         ΦPn = PhiP*Fn*Ae

#     elif (λrf > λf) and (λrw < λw) :    # Flexural Buckling of Member with Slender Web
#         Fel = (1.31*(λrw/λw))**2*Fy
#         if λw <= λrw*math.sqrt(Fy/Fn) : 
#             Ae = Ag
#         else : 
#             be = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
#             Ae = Ag-(((D-2*tf-2*radi)-be)*tw)

#         ΦPn = PhiP*Fn*Ae

#     else :                              # Flexural Buckling of Member with Slender Flange & Web
#         Felf = (1.49*(λrf/λf))**2*Fy
#         Felw = (1.31*(λrw/λw))**2*Fy
#         if λf <= λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
#             Ae = Ag
#         elif λf > λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
#             be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
#             Ae = Ag-((W-2*be)*tf)*2
#         elif λf <= λrf*math.sqrt(Fy/Fn) and λw > λrw*math.sqrt(Fy/Fn) : 
#             be = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
#             Ae = Ag-(((D-2*tf-2*radi)-be)*tw)
#         else : 
#             be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
#             he = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
#             Ae = (be*2*tf)*2+(he*tw)

#         ΦPn = PhiP*Fn*Ae

#     return ΦPn

def Calculate_Flexural_Buckling(secDict,n,Ly_) : 
    Lz_ = Ly_/n
    Ag = secDict["Area"]
    Iyy, Izz = secDict["Iyy"], secDict["Izz"]
    ry, rz = secDict["ry"], secDict["rz"]
    J, Cwp = secDict["J"], secDict["Cwp"]
    D, W = secDict["D"], secDict["W"]
    tw, tf, radi = secDict["tw"], secDict["tf"], secDict["radi"]
    
    SR = max(K*Ly_/ry, K*Lz_/rz)
    Fe1 = (math.pi**2)*E/(SR**2)
    Fe2 = ((math.pi**2)*E*Cwp/((K*Ly_)**2)+77.2*J)/(Iyy+Izz)
    Fe = Fe1
    # print(Fe1,Fe2)

    λrf = 0.56*math.sqrt(E/Fy)
    λrw = 1.49*math.sqrt(E/Fy)
    λf = W/(2*tf)
    λw = (D-2*tf-2*radi)/tw
    
    if SR <= 4.71*math.sqrt(E/(Fy)) : 
        Fn = 0.658**(Fy/Fe)*Fy
    else : 
        Fn = 0.877*Fe


    if (λrf > λf) and (λrw > λw) :      # Flexural Buckling of Member without Slender Element
        ΦPn = PhiP*Fn*Ag

    elif (λrf < λf) and (λrw > λw) :    # Flexural Buckling of Member with Slender Flange
        Fel = (1.49*(λrf/λf))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-((W-2*be)*tf)*2
        
        ΦPn = PhiP*Fn*Ae

    elif (λrf > λf) and (λrw < λw) :    # Flexural Buckling of Member with Slender Web
        Fel = (1.31*(λrw/λw))**2*Fy
        if λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        else : 
            be = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Fel/Fn))*math.sqrt(Fel/Fn)
            Ae = Ag-(((D-2*tf-2*radi)-be)*tw)

        ΦPn = PhiP*Fn*Ae

    else :                              # Flexural Buckling of Member with Slender Flange & Web
        Felf = (1.49*(λrf/λf))**2*Fy
        Felw = (1.31*(λrw/λw))**2*Fy
        if λf <= λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            Ae = Ag
        elif λf > λrf*math.sqrt(Fy/Fn) and λw <= λrw*math.sqrt(Fy/Fn) : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            Ae = Ag-((W-2*be)*tf)*2
        elif λf <= λrf*math.sqrt(Fy/Fn) and λw > λrw*math.sqrt(Fy/Fn) : 
            be = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = Ag-(((D-2*tf-2*radi)-be)*tw)
        else : 
            be = (W/2)*(1-0.22*math.sqrt(Felf/Fn))*math.sqrt(Felf/Fn)
            he = (D-2*tf-2*radi)*(1-0.18*math.sqrt(Felw/Fn))*math.sqrt(Felw/Fn)
            Ae = (be*2*tf)*2+(he*tw)

        ΦPn = PhiP*Fn*Ae

    return ΦPn

Calculate_Flexural_Buckling(Sections[2],2,5000)

706.3614932710938

In [9]:
# def Calculate_Lateral_Torsional_Buckling(List,n,Ly_) : 
#     Syy = List[8]
#     Zyy = List[10]
#     Izz = List[13]
#     rz = List[16]
#     J = List[17]
#     Cwp = List[18]
#     D = List[20]
#     W = List[21]
#     tw = List[22]
#     tf = List[23]
#     radi = List[24]
#     Mp = Fy*Zyy
#     Cb = 1.0

#     λw = (D-2*tf-2*radi)/tw
#     λpw = 3.76*math.sqrt(E/Fy)
#     λrw = 5.70*math.sqrt(E/Fy)

#     if n == 0 :
#         Lz_ = 0
#     else : 
#         Lz_ = Ly_/n

#     if λw <= λpw :                                       # Compact Web & Compact/Non-Compact/Slender Flange
#         Lb = Lz_
#         rts = math.sqrt(math.sqrt(Izz*Cwp)/Syy)
#         Lp = 1.76*rz*math.sqrt(E/Fy)
#         Lr = 1.95*rts*E/(0.7*Fy)*math.sqrt(J/(Syy*(D-tf))+math.sqrt((J/(Syy*(D-tf)))**2+6.76*(0.7*Fy/E)**2))
#         Fcr = Cb*math.pi**2*E/(Lb/rts)**2*math.sqrt(1+0.078*J/(Syy*(D-tf))*(Lb/rts)**2)

#         if Lb <= Lp : 
#             Mny = Mp
#         elif Lp < Lb <= Lr : 
#             Mny = min(Cb*(Mp-(Mp-0.7*Fy*Syy)*(Lb-Lp)/(Lr-Lp)),Mp)
#         elif Lb > Lr : 
#             Mny = min(Fcr*Syy,Mp)

#     elif λpw < λw <= λrw :                                # Non-Compact Web & Compact/Non-Compact/Slender Flange
#         Lb = Lz_
#         Izc = (tf*W**3)/12
#         Syc = Syy
#         Myc = Fy * Syc
#         if Izc/Izz > 0.23 : 
#             if λw <= λpw : 
#                 Rpc = Mp/Myc
#                 J_ = J
#             else : 
#                 Rpc = min((Mp/Myc-(Mp/Myc-1)*(λw-λpw)/(λrw-λpw)),Mp-Myc)
#                 J_ = J
#         else : 
#             Rpc = 1
#             J_ = 0

#         FL = 0.7*Fy
#         aw = ((D-2*tf-2*radi)*tw)/(W*tf)
#         rt = W/math.sqrt(12*(1+1/6*aw))
#         Lp = 1.1*rt*math.sqrt(E/Fy)
#         Lr = 1.95*rt*(E/FL)*math.sqrt(J_/(Syy*(D-tf))+math.sqrt((J_/(Syy*(D-tf)))**2+6.76*(0.7*FL/E)**2))

#         Fcr = Cb*math.pi**2*E/(Lb/rt)**2*math.sqrt(1+0.078*J_/(Syy*(D-tf))*(Lb/rt)**2)

#         if Lb <= Lp : 
#             Mny = Rpc*Myc
#         elif Lp < Lb <= Lr : 
#             Mny = min(Cb*(Rpc*Myc-(Rpc*Myc-0.7*FL*Syy)*(Lb-Lp)/(Lr-Lp)),Rpc*Myc)
#         elif Lb > Lr : 
#             Mny = min(Fcr*Syy,Rpc*Myc)

#     else :                                                 # Slender Web & Compact/Non-Compact/Slender Flange
#         Lb = Lz_
#         Syc = Syy
#         aw = ((D-2*tf-2*radi)*tw)/(W*tf)
#         rt = W/math.sqrt(12*(1+1/6*aw))
#         Lp = 1.1*rt*math.sqrt(E/Fy)
#         Lr = math.pi*rt*math.sqrt(E/(0.7*Fy))
#         Rpg = min(1-(aw/(1200+300*aw))*((D-2*tf-2*radi)/tw-5.7*math.sqrt(E/Fy)),1)

#         if Lb <= Lp : 
#             Mny = Rpg*Fy*Syc
#         elif Lp < Lb <= Lr : 
#             Fcr = Cb*(Fy-(0.3*Fy)*(Lb-Lp)/(Lr-Lp))
#             Mny = min(Rpg*Fcr*Syc,Fy)
#         elif Lb > Lr : 
#             Fcr = Cb*math.pi**2*E/(Lb/((W/2)/math.sqrt(12*(1+1/6*(((D-2*tf-2*radi)*tw)/((W/2)*tf))))))**2
#             Mny = min(Rpg*Fcr*Syc,Fy)

#     ΦMny = PhiM*Mny

#     return ΦMny

def Calculate_Lateral_Torsional_Buckling(secDict,n,Ly_) : 
    Syy, Zyy, Izz, rz = secDict["Syy"], secDict["Zyy"], secDict["Izz"], secDict["rz"]
    J, Cwp, D, W = secDict["J"], secDict["Cwp"], secDict["D"], secDict["W"]
    tw, tf, radi = secDict["tw"], secDict["tf"], secDict["radi"]
    Mp = Fy*Zyy
    Cb = 1.0

    λw = (D-2*tf-2*radi)/tw
    λpw = 3.76*math.sqrt(E/Fy)
    λrw = 5.70*math.sqrt(E/Fy)

    if n == 0 :
        Lz_ = 0
    else : 
        Lz_ = Ly_/n

    if λw <= λpw :                                       # Compact Web & Compact/Non-Compact/Slender Flange
        Lb = Lz_
        rts = math.sqrt(math.sqrt(Izz*Cwp)/Syy)
        Lp = 1.76*rz*math.sqrt(E/Fy)
        Lr = 1.95*rts*E/(0.7*Fy)*math.sqrt(J/(Syy*(D-tf))+math.sqrt((J/(Syy*(D-tf)))**2+6.76*(0.7*Fy/E)**2))
        Fcr = Cb*math.pi**2*E/(Lb/rts)**2*math.sqrt(1+0.078*J/(Syy*(D-tf))*(Lb/rts)**2)

        if Lb <= Lp : 
            Mny = Mp
        elif Lp < Lb <= Lr : 
            Mny = min(Cb*(Mp-(Mp-0.7*Fy*Syy)*(Lb-Lp)/(Lr-Lp)),Mp)
        elif Lb > Lr : 
            Mny = min(Fcr*Syy,Mp)

    elif λpw < λw <= λrw :                                # Non-Compact Web & Compact/Non-Compact/Slender Flange
        Lb = Lz_
        Izc = (tf*W**3)/12
        Syc = Syy
        Myc = Fy * Syc
        if Izc/Izz > 0.23 : 
            if λw <= λpw : 
                Rpc = Mp/Myc
                J_ = J
            else : 
                Rpc = min((Mp/Myc-(Mp/Myc-1)*(λw-λpw)/(λrw-λpw)),Mp-Myc)
                J_ = J
        else : 
            Rpc = 1
            J_ = 0

        FL = 0.7*Fy
        aw = ((D-2*tf-2*radi)*tw)/(W*tf)
        rt = W/math.sqrt(12*(1+1/6*aw))
        Lp = 1.1*rt*math.sqrt(E/Fy)
        Lr = 1.95*rt*(E/FL)*math.sqrt(J_/(Syy*(D-tf))+math.sqrt((J_/(Syy*(D-tf)))**2+6.76*(0.7*FL/E)**2))

        Fcr = Cb*math.pi**2*E/(Lb/rt)**2*math.sqrt(1+0.078*J_/(Syy*(D-tf))*(Lb/rt)**2)

        if Lb <= Lp : 
            Mny = Rpc*Myc
        elif Lp < Lb <= Lr : 
            Mny = min(Cb*(Rpc*Myc-(Rpc*Myc-0.7*FL*Syy)*(Lb-Lp)/(Lr-Lp)),Rpc*Myc)
        elif Lb > Lr : 
            Mny = min(Fcr*Syy,Rpc*Myc)

    else :                                                 # Slender Web & Compact/Non-Compact/Slender Flange
        Lb = Lz_
        Syc = Syy
        aw = ((D-2*tf-2*radi)*tw)/(W*tf)
        rt = W/math.sqrt(12*(1+1/6*aw))
        Lp = 1.1*rt*math.sqrt(E/Fy)
        Lr = math.pi*rt*math.sqrt(E/(0.7*Fy))
        Rpg = min(1-(aw/(1200+300*aw))*((D-2*tf-2*radi)/tw-5.7*math.sqrt(E/Fy)),1)

        if Lb <= Lp : 
            Mny = Rpg*Fy*Syc
        elif Lp < Lb <= Lr : 
            Fcr = Cb*(Fy-(0.3*Fy)*(Lb-Lp)/(Lr-Lp))
            Mny = min(Rpg*Fcr*Syc,Fy)
        elif Lb > Lr : 
            Fcr = Cb*math.pi**2*E/(Lb/((W/2)/math.sqrt(12*(1+1/6*(((D-2*tf-2*radi)*tw)/((W/2)*tf))))))**2
            Mny = min(Rpg*Fcr*Syc,Fy)

    ΦMny = PhiM*Mny

    return ΦMny



In [10]:
# def Calculate_Compression_Flange_Local_Buckling(List) : 
#     Syy = List[8]
#     Zyy = List[10]
#     Izz = List[13]
#     D = List[20]
#     W = List[21]
#     tw = List[22]
#     tf = List[23]
#     radi = List[24]
#     Mp = Fy * Zyy

#     λf = W/(2*tf)
#     λw = (D-2*tf-2*radi)/tw
#     λpf = 0.38*math.sqrt(E/Fy)
#     λpw = 3.76*math.sqrt(E/Fy)
#     λrf = 1.00*math.sqrt(E/Fy)
#     λrw = 5.70*math.sqrt(E/Fy)

#     if λw <= λpw :
#         if λf <= λpf :                                                   # Compact Web & Compact Flange
#             Mny = Mp
#         elif λpf < λf <= λrf :                                           # Compact Web & Non-Compact Flange
#             Mny = Mp-(Mp-0.7*Fy*Syy)*(λf-λpf)/(λrf-λpf)
#         elif λrf < λf :                                                  # Compact Web & Slender Flange
#             kc = min(max(4/(math.sqrt((D-2*tf-2*radi)/tw)),0.35),0.76)
#             Mny = 0.9*E*kc*Syy/λf**2


#     elif λpw < λw <= λrw :
#         Izc = (tf*D**3)/12
#         Syc = Syy
#         Myc = Fy * Syc
#         FL = 0.7*Fy
#         if Izc/Izz > 0.23 : 
#             if λw <= λpw : 
#                 Rpc = Mp/Myc
#             else : 
#                 Rpc = min((Mp/Myc-(Mp/Myc-1)*(λw-λpw)/(λrw-λpw)),Mp-Myc)
#         else : 
#             Rpc = 1

#         if λf <= λpf :                                                   # Non-Compact Web & Compact Flange
#             Mny = Rpc*Myc
#         elif λpf < λf <= λrf :                                           # Non-Compact Web & Non-Compact Flange
#             Mny = Rpc*Myc-(Rpc*Myc-FL*Syc)*(λf-λpf)/(λrf-λpf)
#         elif λrf < λf :                                                  # Non-Compact Web & Slender Flange
#             kc = min(max(4/(math.sqrt((D-2*tf-2*radi)/tw)),0.35),0.76)
#             Mny = 0.9*E*kc*Syc/λf**2

#     elif λrw < λw : 
#         Syc = Syy
#         aw = ((D-2*tf-2*radi)*tw)/((W/2)*tf)
#         Rpg = min(1-(aw/(1200+300*aw))*((D-2*tf-2*radi)/tw-5.7*math.sqrt(E/Fy)),1)

#         if λf <= λpf :                                                   # Slender Web & Compact Flange
#             Mny = Rpg*Fy*Syc
#         elif λpf < λf <= λrf :                                           # Slender Web & Non-Compact Flange
#             Fcr = Fy-(0.3*Fy)*(λf-λpf)/(λrf-λpf)
#             Mny = Rpg*Fcr*Syc
#         elif λrf < λf :                                                  # Slender Web & Slender Flange
#             kc = min(max(4/(math.sqrt((D-2*tf-2*radi)/tw)),0.35),0.76)
#             Fcr = 0.9*E*kc/λf**2
#             Mny = Rpg*Fcr*Syc
    
#     ΦMny = PhiM*Mny

#     return ΦMny

def Calculate_Compression_Flange_Local_Buckling(secDict) :     
    Syy, Zyy, Izz, rz = secDict["Syy"], secDict["Zyy"], secDict["Izz"], secDict["rz"]
    J, Cwp, D, W = secDict["J"], secDict["Cwp"], secDict["D"], secDict["W"]
    tw, tf, radi = secDict["tw"], secDict["tf"], secDict["radi"]
    Mp = Fy*Zyy

    λf = W/(2*tf)
    λw = (D-2*tf-2*radi)/tw
    λpf = 0.38*math.sqrt(E/Fy)
    λpw = 3.76*math.sqrt(E/Fy)
    λrf = 1.00*math.sqrt(E/Fy)
    λrw = 5.70*math.sqrt(E/Fy)

    if λw <= λpw :
        if λf <= λpf :                                                   # Compact Web & Compact Flange
            Mny = Mp
        elif λpf < λf <= λrf :                                           # Compact Web & Non-Compact Flange
            Mny = Mp-(Mp-0.7*Fy*Syy)*(λf-λpf)/(λrf-λpf)
        elif λrf < λf :                                                  # Compact Web & Slender Flange
            kc = min(max(4/(math.sqrt((D-2*tf-2*radi)/tw)),0.35),0.76)
            Mny = 0.9*E*kc*Syy/λf**2


    elif λpw < λw <= λrw :
        Izc = (tf*D**3)/12
        Syc = Syy
        Myc = Fy * Syc
        FL = 0.7*Fy
        if Izc/Izz > 0.23 : 
            if λw <= λpw : 
                Rpc = Mp/Myc
            else : 
                Rpc = min((Mp/Myc-(Mp/Myc-1)*(λw-λpw)/(λrw-λpw)),Mp-Myc)
        else : 
            Rpc = 1

        if λf <= λpf :                                                   # Non-Compact Web & Compact Flange
            Mny = Rpc*Myc
        elif λpf < λf <= λrf :                                           # Non-Compact Web & Non-Compact Flange
            Mny = Rpc*Myc-(Rpc*Myc-FL*Syc)*(λf-λpf)/(λrf-λpf)
        elif λrf < λf :                                                  # Non-Compact Web & Slender Flange
            kc = min(max(4/(math.sqrt((D-2*tf-2*radi)/tw)),0.35),0.76)
            Mny = 0.9*E*kc*Syc/λf**2

    elif λrw < λw : 
        Syc = Syy
        aw = ((D-2*tf-2*radi)*tw)/((W/2)*tf)
        Rpg = min(1-(aw/(1200+300*aw))*((D-2*tf-2*radi)/tw-5.7*math.sqrt(E/Fy)),1)

        if λf <= λpf :                                                   # Slender Web & Compact Flange
            Mny = Rpg*Fy*Syc
        elif λpf < λf <= λrf :                                           # Slender Web & Non-Compact Flange
            Fcr = Fy-(0.3*Fy)*(λf-λpf)/(λrf-λpf)
            Mny = Rpg*Fcr*Syc
        elif λrf < λf :                                                  # Slender Web & Slender Flange
            kc = min(max(4/(math.sqrt((D-2*tf-2*radi)/tw)),0.35),0.76)
            Fcr = 0.9*E*kc/λf**2
            Mny = Rpg*Fcr*Syc
    
    ΦMny = PhiM*Mny

    return ΦMny

In [11]:
# def Calculate_Tension_Flange_Yielding(List) : 
#     Zyy = List[10]
#     D = List[20]
#     tw = List[22]
#     tf = List[23]
#     radi = List[24]
#     Mp = Fy * Zyy

#     λw = (D-2*tf-2*radi)/tw
#     λpw = 3.76*math.sqrt(E/Fy)
#     λrw = 5.70*math.sqrt(E/Fy)

#     if λw <= λpw :
#         Mny = Mp                                                            # Compact Web & Compact/Non-Compact/Slender Flange

#     elif λpw < λw <= λrw :
#         Mny = Mp                                                            # Non-Compact Web & Compact/Non-Compact/Slender Flange

#     elif λrw < λw : 
#         Mny = Mp                                                            # Slender Web & Compact/Non-Compact/Slender Flange

#     ΦMny = PhiM*Mny

#     return ΦMny

def Calculate_Tension_Flange_Yielding(secDict) :     
    Syy, Zyy, Izz, rz = secDict["Syy"], secDict["Zyy"], secDict["Izz"], secDict["rz"]
    J, Cwp, D, W = secDict["J"], secDict["Cwp"], secDict["D"], secDict["W"]
    tw, tf, radi = secDict["tw"], secDict["tf"], secDict["radi"]
    Mp = Fy*Zyy

    λw = (D-2*tf-2*radi)/tw
    λpw = 3.76*math.sqrt(E/Fy)
    λrw = 5.70*math.sqrt(E/Fy)

    if λw <= λpw :
        Mny = Mp                                                            # Compact Web & Compact/Non-Compact/Slender Flange

    elif λpw < λw <= λrw :
        Mny = Mp                                                            # Non-Compact Web & Compact/Non-Compact/Slender Flange

    elif λrw < λw : 
        Mny = Mp                                                            # Slender Web & Compact/Non-Compact/Slender Flange

    ΦMny = PhiM*Mny

    return ΦMny

In [12]:
# def Calculate_Minor_Axis_Flange_Local_Buckling(List) :
#     Szz = List[9]
#     Zzz = List[11]
#     W = List[21]
#     tf = List[23]
#     Mp = Fy*Zzz

#     λf = W/(2*tf)
#     λpf = 0.38*math.sqrt(E/Fy)
#     λrf = 1.00*math.sqrt(E/Fy)

#     if λf <= λpf :                                                   # Compact Flange
#         Mnz = min(Fy*Zzz, 1.6*Fy*Szz)
#     elif λpf < λf <= λrf :                                           # Non-Compact Flange
#         Mnz = Mp-(Mp-0.7*Fy*Szz)*(λf-λpf)/(λrf-λpf)
#     elif λrf < λf :                                                  # lender Flange
#         Fcr = 0.7*E/λf**2
#         Mnz = Fcr*Szz

#     ΦMnz = PhiM*Mnz

#     return ΦMnz

def Calculate_Minor_Axis_Flange_Local_Buckling(secDict) :    
    Szz, Zzz, Zyy, rz = secDict["Szz"], secDict["Zzz"], secDict["Zyy"], secDict["rz"]
    J, Cwp, D, W = secDict["J"], secDict["Cwp"], secDict["D"], secDict["W"]
    tw, tf, radi = secDict["tw"], secDict["tf"], secDict["radi"]
    Mp = Fy*Zyy

    λf = W/(2*tf)
    λpf = 0.38*math.sqrt(E/Fy)
    λrf = 1.00*math.sqrt(E/Fy)

    if λf <= λpf :                                                   # Compact Flange
        Mnz = min(Fy*Zzz, 1.6*Fy*Szz)
    elif λpf < λf <= λrf :                                           # Non-Compact Flange
        Mnz = Mp-(Mp-0.7*Fy*Szz)*(λf-λpf)/(λrf-λpf)
    elif λrf < λf :                                                  # lender Flange
        Fcr = 0.7*E/λf**2
        Mnz = Fcr*Szz

    ΦMnz = PhiM*Mnz

    return ΦMnz

In [13]:
# def Calculate_Shear_Capacity(List) : 
#     D = List[20]
#     W = List[21]
#     tw = List[22]
#     tf = List[23]
#     radi = List[24]
#     λw = (D-2*tf-2*radi)/tw
#     λf = W/(2*tf)

#     if λw <= 2.24*math.sqrt(E/Fy) : 
#         Cv1 = 1.0
#         PhiV = 1.0
#     else : 
#         if λw <= 1.1*math.sqrt(5.34*E/Fy) : 
#             Cv1 = 1.0
#             PhiV = 0.9
#         else : 
#             Cv1 = 1.1*math.sqrt(5.34*E/Fy)/λw
#             PhiV = 0.9

#     ΦVnz = PhiV*0.6*Fy*tw*D*Cv1      # Shear Strength of Webs

#     if λf <= 1.1*math.sqrt(1.2*E/Fy) : 
#         Cv2 = 1.0
#         PhiV = 0.9
#     elif 1.1*math.sqrt(1.2*E/Fy) < λf <= 1.37*math.sqrt(1.2*E/Fy) :
#         Cv2 = 1.1*math.sqrt(1.2*E/Fy)/λw
#         PhiV = 0.9
#     else : 
#         Cv2 = 1.51*1.2*E/(λw**2*Fy)
#         PhiV = 0.9

#     ΦVny = PhiV*0.6*Fy*W*tf*Cv2*2    # Shear Strength of Flanges

#     return ΦVnz, ΦVny

def Calculate_Shear_Capacity(secDict) : 
    D,W = secDict["D"], secDict["W"], 
    tw,tf,radi = secDict["tw"], secDict["tf"], secDict["radi"]
    λw = (D-2*tf-2*radi)/tw
    λf = W/(2*tf)

    if λw <= 2.24*math.sqrt(E/Fy) : 
        Cv1 = 1.0
        PhiV = 1.0
    else : 
        if λw <= 1.1*math.sqrt(5.34*E/Fy) : 
            Cv1 = 1.0
            PhiV = 0.9
        else : 
            Cv1 = 1.1*math.sqrt(5.34*E/Fy)/λw
            PhiV = 0.9

    ΦVnz = PhiV*0.6*Fy*tw*D*Cv1      # Shear Strength of Webs

    if λf <= 1.1*math.sqrt(1.2*E/Fy) : 
        Cv2 = 1.0
        PhiV = 0.9
    elif 1.1*math.sqrt(1.2*E/Fy) < λf <= 1.37*math.sqrt(1.2*E/Fy) :
        Cv2 = 1.1*math.sqrt(1.2*E/Fy)/λw
        PhiV = 0.9
    else : 
        Cv2 = 1.51*1.2*E/(λw**2*Fy)
        PhiV = 0.9

    ΦVny = PhiV*0.6*Fy*W*tf*Cv2*2    # Shear Strength of Flanges

    return ΦVnz, ΦVny

In [14]:
# def Calculate_Combined_Ratio(List, Pc, Mcy, Mcz) : 
#     if List[0]/Pc >= 0.2 : 
#         CR = List[0]/Pc+(8/9)*(List[1]/Mcy+List[2]/Mcz)
#     else : 
#         CR = List[0]/(2*Pc)+(List[1]/Mcy+List[2]/Mcz)
#     return CR

def Calculate_Combined_Ratio(forceDict, Pc, Mcy, Mcz) : 
    if forceDict["Pu"]/Pc >= 0.2 : 
        CR = forceDict["Pu"]/Pc+(8/9)*(forceDict["Muy"]/Mcy+forceDict["Muz"]/Mcz)
    else : 
        CR = forceDict["Pu"]/(2*Pc)+(forceDict["Muy"]/Mcy+forceDict["Muz"]/Mcz)
    return CR

In [15]:
# Column_List = copy.deepcopy(Section)

# def Calculate_Column_Capacity(List) : 
#     res = []
#     n = Column_Unbraced_Segment
#     SR = Calculate_Slenderness_Ratio(List,n,Column_Length)
#     ΦPn1 = Calculate_Tensile_Strength(List)
#     ΦPn2 = Calculate_Flexural_Buckling(List,n,Column_Length)
#     if Column_Data[0] > 0 : 
#         ΦPn = ΦPn1
#     else : 
#         ΦPn = ΦPn2
#     ΦMny1 = Calculate_Lateral_Torsional_Buckling(List,n,Column_Length)
#     ΦMny2 = Calculate_Compression_Flange_Local_Buckling(List)
#     ΦMny3 = Calculate_Tension_Flange_Yielding(List)
#     ΦMny = min(ΦMny1, ΦMny2, ΦMny3)
#     ΦMnz = Calculate_Minor_Axis_Flange_Local_Buckling(List)
#     ΦVnz, ΦVny = Calculate_Shear_Capacity(List)
#     # CR = Calculate_Combined_Ratio(Column_Data[0], Column_Data[1], Column_Data[2], ΦPn, ΦMny, ΦMnz)
#     CR = Calculate_Combined_Ratio([Column_Data[0], Column_Data[1], Column_Data[2]], ΦPn, ΦMny, ΦMnz)

#     res.append(List[0])
#     res.append(SR)
#     res.append(ΦPn)
#     res.append(ΦMny)
#     res.append(ΦMnz)
#     res.append(ΦVnz)
#     res.append(ΦVny)
#     res.append(abs(Column_Data[0])/ΦPn)
#     res.append(abs(Column_Data[1])/ΦMny)
#     res.append(abs(Column_Data[2])/ΦMnz)
#     res.append(abs(Column_Data[4])/ΦVnz)
#     res.append(abs(Column_Data[3])/ΦVny)
#     res.append(CR)

#     if res[1] < 200 and res[7]<1 and res[8]<1 and res[9]<1 and res[10]<1 and res[11]<1 and res[12]<1 :
#         res.append('OK')
#     else : 
#         res.append('NG')

#     return res

# Column_Calculated_Result = Calculate_Column_Capacity(Column_Section)

# def Select_Applicable_Column(List) :
#     res = []
#     for inner_list in List : 
#         res.append(Calculate_Column_Capacity(inner_list))
        
#     return res

# Calculated_Applicable_Column = Select_Applicable_Column(Column_List)

# def Delete_NG_Element(List) : 
#     res = []
#     for inner_list in List :
#         if inner_list[13] == 'OK' : 
#             res.append(inner_list)
#     return res

# Selected_Column_List = Delete_NG_Element(Calculated_Applicable_Column)

# print(Column_Calculated_Result)       # 적용 부재 Ratio 검토
# print(Selected_Column_List)           # OK 나오는 Column List

In [16]:
Column_Section["forceDict"]

{'Pu': -50.05, 'Muy': 7518, 'Muz': 7308, 'Vuy': 50, 'Vuz': 100}

In [17]:
# Sections = list(map(lambda x: convert_ListToDict(x, propKeys), Section))
Column_List = copy.deepcopy(Sections)

def Calculate_Column_Capacity(secDict) : 
    forceDict = secDict["forceDict"]
    res = []
    n = Column_Unbraced_Segment
    SR = Calculate_Slenderness_Ratio(secDict,n,Column_Length)
    ΦPn1 = Calculate_Tensile_Strength(secDict)
    ΦPn2 = Calculate_Flexural_Buckling(secDict,n,Column_Length)
    if forceDict["Pu"] > 0 : 
        ΦPn = ΦPn1
    else : 
        ΦPn = ΦPn2
    ΦMny1 = Calculate_Lateral_Torsional_Buckling(secDict,n,Column_Length)
    ΦMny2 = Calculate_Compression_Flange_Local_Buckling(secDict)
    ΦMny3 = Calculate_Tension_Flange_Yielding(secDict)
    ΦMny = min(ΦMny1, ΦMny2, ΦMny3)
    ΦMnz = Calculate_Minor_Axis_Flange_Local_Buckling(secDict)
    ΦVnz, ΦVny = Calculate_Shear_Capacity(secDict)
    # CR = Calculate_Combined_Ratio(Column_Data[0], Column_Data[1], Column_Data[2], ΦPn, ΦMny, ΦMnz)
    # CR = Calculate_Combined_Ratio([Column_Data[0], Column_Data[1], Column_Data[2]], ΦPn, ΦMny, ΦMnz)
    CR = Calculate_Combined_Ratio(forceDict, ΦPn, ΦMny, ΦMnz)
    
    ratioDict = {
        "sectionName": secDict["Section_Name"],
        "SR": SR,
        "ΦPn": ΦPn, 
        "ΦMny": ΦMny, 
        "ΦMnz": ΦMnz, 
        "ΦVnz": ΦVnz, 
        "ΦVny": ΦVny,
        "pRatio": abs(forceDict["Pu"])/ΦPn,
        "myRatio": abs(forceDict["Muy"])/ΦMny,
        "mzRatio": abs(forceDict["Muz"])/ΦMnz,
        "vzRatio": abs(forceDict["Vuz"])/ΦVnz,
        "vyRatio": abs(forceDict["Vuy"])/ΦVny,
        "CR": CR,
    }
    secDict["ratioDict"] = ratioDict
    
    if ratioDict["SR"] < 200 and ratioDict["pRatio"]<1 and ratioDict["myRatio"]<1 and ratioDict["mzRatio"]<1 and ratioDict["vzRatio"]<1 and ratioDict["vyRatio"]<1 and ratioDict["CR"]<1 :
        secDict["chk_result"] = 'OK'
    else : 
        secDict["chk_result"] = 'NG'

    return secDict

Column_Calculated_Result = Calculate_Column_Capacity(Column_Section)
Column_Calculated_Result
# secDicts
      # OK 나오는 Column List

{'Section_Name': 'UB-533x210x82',
 'Area': 10500.0,
 'Asy': 3673.12,
 'Asz': 5071.68,
 'Ybar': 104.35,
 'Zbar': 264.15,
 'Qyb': 105395.0,
 'Qzb': 5444.46,
 'Syy': 1799000.0,
 'Szz': 192000.0,
 'Zyy': 2058000.0,
 'Zzz': 300000,
 'Iyy': 475200000.0,
 'Izz': 20040000.0,
 'Iyz': 0,
 'ry': 213.0,
 'rz': 43.8,
 'J': 515000.0,
 'Cwp': 1329290000000,
 'SW': 82,
 'D': 528.3,
 'W': 208.7,
 'tw': 9.6,
 'tf': 13.2,
 'radi': 12.7,
 'forceDict': {'Pu': -50.05, 'Muy': 7518, 'Muz': 7308, 'Vuy': 50, 'Vuz': 100},
 'ratioDict': {'sectionName': 'UB-533x210x82',
  'SR': 79.90867579908677,
  'ΦPn': 1779.3988359676994,
  'ΦMny': 430771.6142873125,
  'ΦMnz': 71550.0,
  'ΦVnz': 806.39712,
  'ΦVny': 788.435208,
  'pRatio': 0.028127477094128286,
  'myRatio': 0.017452403432937684,
  'mzRatio': 0.10213836477987422,
  'vzRatio': 0.12400837939500578,
  'vyRatio': 0.06341675193175797,
  'CR': 0.10552702966574776},
 'chk_result': 'OK'}

In [18]:
def Select_Applicable_Column(secDicts) :
    res = []
    for secDict in secDicts : 
        res.append(Calculate_Column_Capacity(secDict))
        
    return res

Calculated_Applicable_Column = Select_Applicable_Column(Columns_secDicts)

def Delete_NG_Element(secDicts) : 
    res = []
    for secDict in secDicts :
        if secDict["chk_result"] == 'OK' : 
            res.append(secDict)
    return res

Selected_Column_List = Delete_NG_Element(Calculated_Applicable_Column)

print(Column_Calculated_Result)       # 적용 부재 Ratio 검토
print(Selected_Column_List)     

{'Section_Name': 'UB-533x210x82', 'Area': 10500.0, 'Asy': 3673.12, 'Asz': 5071.68, 'Ybar': 104.35, 'Zbar': 264.15, 'Qyb': 105395.0, 'Qzb': 5444.46, 'Syy': 1799000.0, 'Szz': 192000.0, 'Zyy': 2058000.0, 'Zzz': 300000, 'Iyy': 475200000.0, 'Izz': 20040000.0, 'Iyz': 0, 'ry': 213.0, 'rz': 43.8, 'J': 515000.0, 'Cwp': 1329290000000, 'SW': 82, 'D': 528.3, 'W': 208.7, 'tw': 9.6, 'tf': 13.2, 'radi': 12.7, 'forceDict': {'Pu': -50.05, 'Muy': 7518, 'Muz': 7308, 'Vuy': 50, 'Vuz': 100}, 'ratioDict': {'sectionName': 'UB-533x210x82', 'SR': 79.90867579908677, 'ΦPn': 1779.3988359676994, 'ΦMny': 430771.6142873125, 'ΦMnz': 71550.0, 'ΦVnz': 806.39712, 'ΦVny': 788.435208, 'pRatio': 0.028127477094128286, 'myRatio': 0.017452403432937684, 'mzRatio': 0.10213836477987422, 'vzRatio': 0.12400837939500578, 'vyRatio': 0.06341675193175797, 'CR': 0.10552702966574776}, 'chk_result': 'OK'}
[{'Section_Name': 'UB-203x102x23', 'Area': 2900.0, 'Asy': 1259.84, 'Asz': 1056.64, 'Ybar': 50.8, 'Zbar': 101.6, 'Qyb': 21876.2, 'Qzb':

In [19]:
# Girder_List = copy.deepcopy(Section)
# Beam_List = copy.deepcopy(Section)

# def Calculate_Girder_Beam_Capacity(List, Element) : 
#     res = []
#     if Element == 'Girder' :
#         n = Girder_Unbraced_Segment
#         SR = Calculate_Slenderness_Ratio(List,n,Girder_Length)
#         ΦPn1 = Calculate_Tensile_Strength(List)
#         ΦPn2 = Calculate_Flexural_Buckling(List,n,Girder_Length)
#         if Girder_Data[0] > 0 : 
#             ΦPn = ΦPn1
#         else : 
#             ΦPn = ΦPn2
#         ΦMny1 = Calculate_Lateral_Torsional_Buckling(List,n,Girder_Length)
#         ΦMny2 = Calculate_Compression_Flange_Local_Buckling(List)
#         ΦMny3 = Calculate_Tension_Flange_Yielding(List)
#         ΦMny = min(ΦMny1, ΦMny2, ΦMny3)
#         ΦMnz = Calculate_Minor_Axis_Flange_Local_Buckling(List)
#         ΦVnz, ΦVny = Calculate_Shear_Capacity(List)
#         CR = Calculate_Combined_Ratio([Girder_Data[0], Girder_Data[1], Girder_Data[2]], ΦPn, ΦMny, ΦMnz)

#         res.append(List[0])
#         res.append(SR)
#         res.append(ΦPn)
#         res.append(ΦMny)
#         res.append(ΦMnz)
#         res.append(ΦVnz)
#         res.append(ΦVny)
#         res.append(abs(Girder_Data[0])/ΦPn)
#         res.append(abs(Girder_Data[0])/ΦMny)
#         res.append(abs(Girder_Data[0])/ΦMnz)
#         res.append(abs(Girder_Data[0])/ΦVnz)
#         res.append(abs(Girder_Data[0])/ΦVny)
#         res.append(CR)

#         if res[1] < 200 and res[7]<1 and res[8]<1 and res[9]<1 and res[10]<1 and res[11]<1 and res[12]<1 :
#             res.append('OK')
#         else : 
#             res.append('NG')

#     if Element == 'Beam' : 
#         n = 1
#         SR = Calculate_Slenderness_Ratio(List,n,Beam_Length)
#         ## Brace 용도 / Start
#         Lbr = Beam_Attached_Girder_Length/Girder_Unbraced_Segment
#         Pr = Girder_Data[0]
#         Mr = Girder_Data[2]
#         Cd = 2
#         ho = Girder_Section[20]
#         Pbr = 0.01*Pr+0.02*(Mr*Cd/ho)
#         βbr = 1/0.75*(8*Pr/Lbr)+1/0.75*(10*Mr*Cd/(Lbr*ho))
#         Mbr = 2*Beam_Length*Pbr/4
#         Stiffness = 48*E*List[12]/(Beam_Attached_Girder_Length*2)**3
#         Stiffness_Ratio = βbr/Stiffness
#         ## Brace 용도 / End
#         ΦPn1 = Calculate_Tensile_Strength(List)
#         ΦPn2 = Calculate_Flexural_Buckling(List,n,Beam_Length)
#         if Beam_Data[0] > 0 : 
#             ΦPn = ΦPn1
#         else : 
#             ΦPn = ΦPn2
#         ΦMny1 = Calculate_Lateral_Torsional_Buckling(List,n,Beam_Length)
#         ΦMny2 = Calculate_Compression_Flange_Local_Buckling(List)
#         ΦMny3 = Calculate_Tension_Flange_Yielding(List)
#         ΦMny = min(ΦMny1, ΦMny2, ΦMny3)
#         ΦMnz = Calculate_Minor_Axis_Flange_Local_Buckling(List)
#         ΦVnz, ΦVny = Calculate_Shear_Capacity(List)
#         # CR = Calculate_Combined_Ratio(Beam_Data[0], Beam_Data[1], Beam_Data[2], ΦPn, ΦMny, ΦMnz)
#         CR = Calculate_Combined_Ratio([Beam_Data[0], Beam_Data[1], Beam_Data[2]], ΦPn, ΦMny, ΦMnz)

#         res.append(List[0])
#         res.append(SR)
#         res.append(ΦPn)
#         res.append(ΦMny)
#         res.append(ΦMnz)
#         res.append(ΦVnz)
#         res.append(ΦVny)
#         res.append(max(abs(Beam_Data[0]),Pbr)/ΦPn)
#         res.append(max(abs(Beam_Data[0]),Mbr)/ΦMny)
#         res.append(abs(Beam_Data[0])/ΦMnz)
#         res.append(abs(Beam_Data[0])/ΦVnz)
#         res.append(abs(Beam_Data[0])/ΦVny)
#         res.append(CR)

#         if res[1] < 200 and res[7]<1 and res[8]<1 and res[9]<1 and res[10]<1 and res[11]<1 and res[12]<1 and Stiffness_Ratio<1 :
#             res.append('OK')
#         else : 
#             res.append('NG')

#     return res

# Girder_Calculated_Result = Calculate_Girder_Beam_Capacity(Girder_Section, 'Girder')
# Beam_Calculated_Result = Calculate_Girder_Beam_Capacity(Beam_Section, 'Beam')

# def Select_Applicable_Girder_Beam(List, Element) :
#     res = []
#     for inner_list in List : 
#         res.append(Calculate_Girder_Beam_Capacity(inner_list, Element))
#     return res

# Calculated_Applicable_Girder = Select_Applicable_Girder_Beam(Girder_List, 'Girder')
# Calculated_Applicable_Beam = Select_Applicable_Girder_Beam(Girder_List, 'Beam')
# Selected_Applicable_Girder = Delete_NG_Element(Calculated_Applicable_Girder)
# Selected_Applicable_Beam = Delete_NG_Element(Calculated_Applicable_Beam)

# print(Girder_Calculated_Result)
# print(Beam_Calculated_Result)
# print(Selected_Applicable_Girder)
# print(Selected_Applicable_Beam)

In [20]:
Girder_List = copy.deepcopy(Section)
Beam_List = copy.deepcopy(Section)

def Calculate_Girder_Beam_Capacity(secDict, elementType) : 
    forceDict = secDict["forceDict"]
    res = []
    if elementType == 'Girder' :
        n = Girder_Unbraced_Segment
        SR = Calculate_Slenderness_Ratio(secDict,n,Girder_Length)
        ΦPn1 = Calculate_Tensile_Strength(secDict)
        ΦPn2 = Calculate_Flexural_Buckling(secDict,n,Girder_Length)
        if forceDict["Pu"] > 0 : 
            ΦPn = ΦPn1
        else : 
            ΦPn = ΦPn2
        ΦMny1 = Calculate_Lateral_Torsional_Buckling(secDict,n,Girder_Length)
        ΦMny2 = Calculate_Compression_Flange_Local_Buckling(secDict)
        ΦMny3 = Calculate_Tension_Flange_Yielding(secDict)
        ΦMny = min(ΦMny1, ΦMny2, ΦMny3)
        ΦMnz = Calculate_Minor_Axis_Flange_Local_Buckling(secDict)
        ΦVnz, ΦVny = Calculate_Shear_Capacity(secDict)
        CR = Calculate_Combined_Ratio(forceDict, ΦPn, ΦMny, ΦMnz)

        ratioDict = {
        "sectionName": secDict["Section_Name"],
        "SR": SR,
        "ΦPn": ΦPn, 
        "ΦMny": ΦMny, 
        "ΦMnz": ΦMnz, 
        "ΦVnz": ΦVnz, 
        "ΦVny": ΦVny,
        "pRatio": abs(forceDict["Pu"])/ΦPn,
        "myRatio": abs(forceDict["Muy"])/ΦMny,
        "mzRatio": abs(forceDict["Muz"])/ΦMnz,
        "vzRatio": abs(forceDict["Vuz"])/ΦVnz,
        "vyRatio": abs(forceDict["Vuy"])/ΦVny,
        "CR": CR,
         }   
        secDict["ratioDict"] = ratioDict
        
        if ratioDict["SR"] < 200 and ratioDict["pRatio"]<1 and ratioDict["myRatio"]<1 and ratioDict["mzRatio"]<1 and ratioDict["vzRatio"]<1 and ratioDict["vyRatio"]<1 and ratioDict["CR"]<1 :
            secDict["chk_result"] = 'OK'
        else : 
            secDict["chk_result"] = 'NG'

    if elementType == 'Beam' : 
        n = 1
        SR = Calculate_Slenderness_Ratio(secDict,n,Beam_Length)
        ## Brace 용도 / Start
        Lbr = Beam_Attached_Girder_Length/Girder_Unbraced_Segment
        Pr = forceDict["Pu"]
        Mr = forceDict["Muz"]
        Cd = 2
        ho = secDict["D"]
        Pbr = 0.01*Pr+0.02*(Mr*Cd/ho)
        βbr = 1/0.75*(8*Pr/Lbr)+1/0.75*(10*Mr*Cd/(Lbr*ho))
        Mbr = 2*Beam_Length*Pbr/4
        Stiffness = 48*E*secDict["Zzz"]/(Beam_Attached_Girder_Length*2)**3
        Stiffness_Ratio = βbr/Stiffness
        ## Brace 용도 / End
        ΦPn1 = Calculate_Tensile_Strength(secDict)
        ΦPn2 = Calculate_Flexural_Buckling(secDict,n,Beam_Length)
        if forceDict["Pu"] > 0 : 
            ΦPn = ΦPn1
        else : 
            ΦPn = ΦPn2
        ΦMny1 = Calculate_Lateral_Torsional_Buckling(secDict,n,Beam_Length)
        ΦMny2 = Calculate_Compression_Flange_Local_Buckling(secDict)
        ΦMny3 = Calculate_Tension_Flange_Yielding(secDict)
        ΦMny = min(ΦMny1, ΦMny2, ΦMny3)
        ΦMnz = Calculate_Minor_Axis_Flange_Local_Buckling(secDict)
        ΦVnz, ΦVny = Calculate_Shear_Capacity(secDict)
        # CR = Calculate_Combined_Ratio(Beam_Data[0], Beam_Data[1], Beam_Data[2], ΦPn, ΦMny, ΦMnz)
        CR = Calculate_Combined_Ratio(forceDict, ΦPn, ΦMny, ΦMnz)

        ratioDict = {
        "sectionName": secDict["Section_Name"],
        "SR": SR,
        "ΦPn": ΦPn, 
        "ΦMny": ΦMny, 
        "ΦMnz": ΦMnz, 
        "ΦVnz": ΦVnz, 
        "ΦVny": ΦVny,
        "pRatio": abs(forceDict["Pu"])/ΦPn,
        "myRatio": abs(forceDict["Muy"])/ΦMny,
        "mzRatio": abs(forceDict["Muz"])/ΦMnz,
        "vzRatio": abs(forceDict["Vuz"])/ΦVnz,
        "vyRatio": abs(forceDict["Vuy"])/ΦVny,
        "CR": CR,
         }   
        secDict["ratioDict"] = ratioDict
        
        if ratioDict["SR"] < 200 and ratioDict["pRatio"]<1 and ratioDict["myRatio"]<1 and ratioDict["mzRatio"]<1 and ratioDict["vzRatio"]<1 and ratioDict["vyRatio"]<1 and ratioDict["CR"]<1 and Stiffness_Ratio<1 :
            secDict["chk_result"] = 'OK'
        else : 
            secDict["chk_result"] = 'NG'

    return secDict

Girder_Calculated_Result = Calculate_Girder_Beam_Capacity(Girder_Section, 'Girder')
Beam_Calculated_Result = Calculate_Girder_Beam_Capacity(Beam_Section, 'Beam')
Girder_Calculated_Result
# def Select_Applicable_Girder_Beam(List, Element) :
#     res = []
#     for inner_list in List : 
#         res.append(Calculate_Girder_Beam_Capacity(inner_list, Element))
#     return res

# Calculated_Applicable_Girder = Select_Applicable_Girder_Beam(Girders_SecDicts, 'Girder')
# Calculated_Applicable_Beam = Select_Applicable_Girder_Beam(Beams_SecDicts, 'Beam')
# Selected_Applicable_Girder = Delete_NG_Element(Calculated_Applicable_Girder)
# Selected_Applicable_Beam = Delete_NG_Element(Calculated_Applicable_Beam)

# print(Girder_Calculated_Result)
# print(Beam_Calculated_Result)
# print(Selected_Applicable_Girder)
# print(Selected_Applicable_Beam)

{'Section_Name': 'UB-254x146x31',
 'Area': 3990.0,
 'Asy': 1675.28,
 'Asz': 1534.15,
 'Ybar': 73.05,
 'Zbar': 125.75,
 'Qyb': 31878.0,
 'Qzb': 2668.15,
 'Syy': 352000.0,
 'Szz': 61300.0,
 'Zyy': 395000.0,
 'Zzz': 94200,
 'Iyy': 44280000.0,
 'Izz': 4480000.0,
 'Iyz': 0,
 'ry': 105.0,
 'rz': 33.5,
 'J': 86500.0,
 'Cwp': 66080500000,
 'SW': 31,
 'D': 251.5,
 'W': 146.1,
 'tw': 6.1,
 'tf': 8.6,
 'radi': 7.6,
 'forceDict': {'Pu': -50.05, 'Muy': 7518, 'Muz': 7308, 'Vuy': 50, 'Vuz': 100},
 'ratioDict': {'sectionName': 'UB-254x146x31',
  'SR': 59.701492537313435,
  'ΦPn': 786.3635067129121,
  'ΦMny': 90752.79472147064,
  'ΦMnz': 22466.7,
  'ΦVnz': 243.92985,
  'ΦVny': 359.59885199999997,
  'pRatio': 0.0636474093377179,
  'myRatio': 0.08284042406708786,
  'mzRatio': 0.3252814164964147,
  'vzRatio': 0.4099539273278773,
  'vyRatio': 0.13904382542355836,
  'CR': 0.37629813589464356},
 'chk_result': 'OK'}

In [21]:
Sub_Column_List = copy.deepcopy(Section)

def Select_Applicable_Sub_Column(List) :
    res = []
    for inner_list in List : 
        n = 1
        to_append = []
        Lbr = Column_Length/Column_Unbraced_Segment
        SR = Calculate_Slenderness_Ratio(inner_list,n,Lbr)
        ΦMny1 = Calculate_Lateral_Torsional_Buckling(inner_list,n,Lbr)
        ΦMny2 = Calculate_Compression_Flange_Local_Buckling(inner_list)
        ΦMny3 = Calculate_Tension_Flange_Yielding(inner_list)
        ΦMny = min(ΦMny1, ΦMny2, ΦMny3)
        ΦMnz = Calculate_Minor_Axis_Flange_Local_Buckling(inner_list)
        ΦVnz, ΦVny = Calculate_Shear_Capacity(inner_list)

        to_append.append(inner_list[0])
        to_append.append(SR)
        to_append.append(ΦMny)
        to_append.append(ΦMnz)
        to_append.append(ΦVnz)
        to_append.append(ΦVny)
        to_append.append(Sub_Column_Data[1]/ΦMny)
        to_append.append(Sub_Column_Data[2]/ΦMnz)
        to_append.append(Sub_Column_Data[4]/ΦVnz)
        to_append.append(Sub_Column_Data[3]/ΦVny)

        if to_append[1] and to_append[6] and to_append[7] and to_append[8] and to_append[9] :
            to_append.append('OK')
            
        else : 
            to_append.append('NG')
        
        to_append.append(inner_list[19])
        res.append(to_append)
        
    return res

Calculated_Applicable_Sub_Column = Select_Applicable_Sub_Column(Sub_Column_List)

def Delete_NG_Sub_Column(List) : 
    res = []
    for inner_list in List :
        if inner_list[10] == 'OK' : 
            res.append(inner_list)
    return res

Selected_Sub_Column_List = Delete_NG_Sub_Column(Calculated_Applicable_Sub_Column)

def Collect_Light_Element(List) : 
    weight_min = min(List, key=lambda x: x[11])
    return weight_min

Lightest_Sub_Column = Collect_Light_Element(Selected_Sub_Column_List)

print(Selected_Sub_Column_List)
print(Lightest_Sub_Column)

TypeError: list indices must be integers or slices, not str

In [ ]:
Brace_List = copy.deepcopy(Section)

def Select_Applicable_Column_Lateral_Brace(List) : # 현재는 Point Brace 만 적용 가능
    res = []
    Lbr = Column_Length/Column_Unbraced_Segment
    Pr = Column_Data[0]
    Mr = Column_Data[2]
    Cd = 2
    ho = Column_Section[20]
    Pbr = 0.01*Pr+0.02*(Mr*Cd/ho)
    βbr = 1/0.75*(8*Pr/Lbr)+1/0.75*(10*Mr*Cd/(Lbr*ho))
    Mbr = 2*Beam_Attached_Girder_Length*Pbr/4
    for inner_list in List : 
        m = 1
        to_append = []
        while True : 
            Stiffness = (48*E*inner_list[12])/(X_Span*2)**3
            SR = Calculate_Slenderness_Ratio(inner_list,m,X_Span)
            ΦPn1 = Calculate_Tensile_Strength(inner_list)
            ΦPn2 = Calculate_Lateral_Torsional_Buckling(inner_list,m,X_Span)
            if Pbr > 0 : 
                ΦPn = ΦPn1
            else : 
                ΦPn = ΦPn2
            ΦMny1 = Calculate_Lateral_Torsional_Buckling(inner_list,m,X_Span)
            ΦMny2 = Calculate_Compression_Flange_Local_Buckling(inner_list)
            ΦMny3 = Calculate_Tension_Flange_Yielding(inner_list)
            ΦMny = min(ΦMny1, ΦMny2, ΦMny3)

            if ΦPn<Pbr or Stiffness<βbr or ΦMny<Mbr or SR > 200 : 
                m = m + 1
            elif ΦPn>Pbr and Stiffness>βbr and ΦMny>Mbr  and SR < 200 :
                m = m
                break
            if (ΦPn<Pbr or Stiffness<βbr or ΦMny<Mbr or SR > 200) and (m > 10) : 
                m = 'NG'
                break

        to_append.append(inner_list[0])
        to_append.append(SR)
        to_append.append(Pbr/ΦPn)
        to_append.append(Mbr/ΦMny)
        to_append.append(βbr/Stiffness)
        to_append.append(m)                     # Lateral Brace의 Unbraced Segment 개수 (Sub-Column 적용 여부)
        res.append(to_append)

    return res

Calculated_Applicable_Column_Lateral_Brace = Select_Applicable_Column_Lateral_Brace(Brace_List)

def Delete_NG_Brace(List) : 
    res = []
    for inner_list in List :
        if isinstance(inner_list[5], int) : 
            res.append(inner_list)
    return res

Selected_Brace_List = Delete_NG_Brace(Calculated_Applicable_Column_Lateral_Brace)

print(Selected_Brace_List)  # Column에 Lateral Brace 적용한다면 적용 가능한 Brace List, Sub-Column 적용 여부

[['UB-305x165x40', 155.84415584415584, 0.001888627776845235, 0.0728535249892957, 0.7728738865161422, 1], ['UB-305x165x54', 152.28426395939087, 0.0014289324528623773, 0.04711594601293605, 0.5653417111719019, 1], ['UB-356x171x51', 155.03875968992247, 0.0015085633635482064, 0.050439047459551745, 0.46672631381352636, 1], ['UB-457x152x60', 185.1851851851852, 0.001285662180543722, 0.04724320126039083, 0.25967955220430383, 1], ['UB-457x191x74', 142.85714285714286, 0.0010247444088876353, 0.024992745686520428, 0.19769203121745535, 1], ['UB-457x191x98', 138.56812933025404, 0.000779625546281713, 0.016888859583673532, 0.1443924973475974, 1], ['UB-533x210x82', 136.986301369863, 0.0009281256503353727, 0.020108040129862895, 0.13907501270201036, 1], ['UB-533x210x101', 131.29102844638948, 0.000755451110738094, 0.014704464186884071, 0.10719942584914084, 1], ['UB-610x229x101', 126.3157894736842, 0.000755451110738094, 0.013488402366949322, 0.08716492486942143, 1], ['UB-610x229x125', 120.72434607645874, 0.